In [1]:
from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [2]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/regions")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [3]:
# Deleting all region data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:BrainRegion 
                OR n:RegionZone 
                OR n:RegionRecord 
                OR n:Nomenclature
                OR n:RegionOther
            DETACH DELETE n
            
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:BrainRegion) ASSERT n.id IS UNIQUE
            DROP INDEX ON :BrainRegion(name)
            DROP CONSTRAINT ON (n:Nomenclature) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Nomenclature(name)
            DROP CONSTRAINT ON (n:RegionOther) ASSERT n.id IS UNIQUE
            DROP INDEX ON :RegionOther(name)
        ''')
        print("wipeddatabase")

wipeddatabase


In [4]:
# Adding regions to graph with BrainRegion label
csv_file_path = "file:///%s" % get_csv_path("regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (d:BrainRegion {id: row.ID, name: row.Region_name, abbreviation: row.Abbreviation, comments: row.Comments})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE CONSTRAINT ON (n:BrainRegion) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :BrainRegion(name)")
    session.run(query)
    get_count_of_type("BrainRegion", session)

Added 30 nodes of type BrainRegion


In [5]:
# Adding region_records to graph with RegionRecord label
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:RegionRecord { id: row.ID, name: row.Region_record_name, coverage: row.Coverage, specificity: row.Specificity, numberOfOriginalRegions: toInt(row.No_original_regions), originalRegionRetained: toInt(row.Original_region_retained), parcellationScheme: row.Parcellation_scheme, atlasCoordinates: row.Atlas_coordinates, illustration: row.Illustration, semanticDescription: row.Semantic_description, annotatedImages: row.Annotated_images, regionalCharacteristics: row.Regional_characteristics, isAtlasRegion: row.Atlas_reg, serialSections: row.Serial_sections, collectorsComment: row.Collectors_comment, documentationScore: toInt(row.Documentation_score)})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("RegionRecord", session)

Added 313 nodes of type RegionRecord


In [6]:
# Adding region_zones to graph with RegionZone label
csv_file_path = "file:///%s" % get_csv_path("region_zones.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:RegionZone { id: row.ID, name: row.Region_zone, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("RegionZone", session)

Added 79 nodes of type RegionZone


In [7]:
# Adding nomenclatures to graph with Nomenclature label
csv_file_path = "file:///%s" % get_csv_path("nomenclatures.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Nomenclature { id: row.ID, name: row.Nomenclature_name, version: row.Version, authors: row.Authors, published: toInt(row.Published), publication_type: row.Publication_type, doi: row.DOI, preferred: 1})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:Nomenclature) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :Nomenclature(name)")
    session.run(query)
    get_count_of_type("Nomenclature", session)

Added 3 nodes of type Nomenclature


In [8]:
# Adding nomebclatures_other to graph with Nomenclature label
csv_file_path = "file:///%s" % get_csv_path("nomenclatures_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Nomenclature { id: row.ID, name: row.Nomenclature_name, version: row.Version, authors: row.Authors, published: toInt(row.Published), publicationType: row.Publication_type, preferred: 0 })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Nomenclature", session)

Added 19 nodes of type Nomenclature


In [10]:
# Adding regions_other to graph with RegionOther label
csv_file_path = "file:///%s" % get_csv_path("regions_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (d:RegionOther {id: row.ID, name: row.Region_name, abbreviation: row.Abbreviation, comments: row.Comments})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE CONSTRAINT ON (n:RegionOther) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :RegionOther(name)")
    session.run(query)
    get_count_of_type("RegionOther", session)

ConstraintError: Node(91807) already exists with label `RegionOther` and property `id` = '6'

In [11]:
# Relationship PRIMARY_REGION between BrainRegion and RegionRecord
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:BrainRegion { id: row.Region})
        MERGE (a)-[:PRIMARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PRIMARY_REGION", session)

Added 624 relationships of type PRIMARY_REGION


In [12]:
# Relationship SECONDARY_REGION between BrainRegion and RegionRecord
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:BrainRegion { id: row.Secondary_region })
        MERGE (a)-[:SECONDARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECONDARY_REGION", session)

Added 58 relationships of type SECONDARY_REGION


In [13]:
# Relationship NOMENCLATURE_SPECIE between Nomenclature and Specie
csv_file_path = "file:///%s" % get_csv_path("nomenclatures.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:NOMENCLATURE_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
csv_file_path = "file:///%s" % get_csv_path("nomenclatures_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:NOMENCLATURE_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NOMENCLATURE_SPECIE", session)

Added 38 relationships of type NOMENCLATURE_SPECIE


In [14]:
# Relationship NOMENCLATURE_STRAIN between Nomenclature and Strain
csv_file_path = "file:///%s" % get_csv_path("nomenclatures.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:NOMENCLATURE_STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
csv_file_path = "file:///%s" % get_csv_path("nomenclatures_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:NOMENCLATURE_STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NOMENCLATURE_STRAIN", session)

Added 34 relationships of type NOMENCLATURE_STRAIN


In [15]:
# Relationship NAMING between All BrainRegion and Nomenclature
csv_file_path = "file:///%s" % get_csv_path("regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:Nomenclature { id: row.Nomenclature })
        MERGE (a)-[:NAMING]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NAMING", session)
    
# Relationship REGION_OTHER_NAMING between All RegionOther and Nomenclature
csv_file_path = "file:///%s" % get_csv_path("regions_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionOther { id: row.ID})
        MATCH (c:Nomenclature { id: row.Nomenclature })
        MERGE (a)-[:REGION_OTHER_NAMING]->(c)
    """ % csv_file_path

csv_file_path = "file:///%s" % get_csv_path("original_region_records.csv")
rel_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionOther {id: row.Original_region})-[r:REGION_OTHER_NAMING]-(b:Nomenclature)
        SET r.originalCoordinateMinAP = row.Original_coord_AP_min
        SET r.originalCoordinateMaxAP = row.Original_coord_AP_max
        SET r.originalCoordinateMinDV = row.Original_coord_DV_min
        SET r.originalCoordinateMaxDV = row.Original_coord_DV_max
        SET r.originalCoordinateMinML = row.Original_coord_ML_min
        SET r.originalCoordinateMaxML = row.Original_coord_ML_max
""" % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_OTHER_NAMING", session)

Added 56 relationships of type NAMING
Added 544 relationships of type REGION_OTHER_NAMING


In [16]:
# Relationship ORIGINAL_FRAMEWORK between Regions_records and Nomenclatures_other
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:Nomenclature { id: row.Original_framework })
        MERGE (a)-[:ORIGINAL_FRAMEWORK]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ORIGINAL_FRAMEWORK", session)

Added 574 relationships of type ORIGINAL_FRAMEWORK


In [17]:
### Add Hierarchical relations between regions
# RAT:
csv_file_path = "file:///%s" % get_csv_path("region_hierarchy_waxholm.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:BrainRegion { id: row.Part_of })
        MERGE (a)-[:PART_OF]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF", session)
    
# MOUSE:
csv_file_path = "file:///%s" % get_csv_path("region_hierarchy_amba.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:BrainRegion { id: row.Part_of })
        MERGE (a)-[:PART_OF]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF", session)


Added 24 relationships of type PART_OF
Added 58 relationships of type PART_OF


In [18]:
# queries to view

q = """
MATCH (n)
WHERE n:BrainRegion OR n:Nomenclature OR n:RegionRecord
RETURN n
"""

In [19]:
## TODO Remove csv_id from all nodes